This notebook verifies that the Bogard ESM dataset I used for geocoding can be joined to the original BogardMapLakes data on GEE. Furthermore, it counts how many of the lakes that were geocoded originally couldn't be matched to PLD (answer: all).

In [10]:
import geopandas as gpd
import pandas as pd

from land_cover.geocode import out_gdb_pth as geocode_pth
from land_cover.load import gee_table_pth

In [3]:
# Load gee table as DataFrame and convert to GeoDataFrame
gee_df = pd.read_csv(gee_table_pth)
gee_gdf = gpd.GeoDataFrame(
    gee_df,
    geometry=gpd.points_from_xy(gee_df['Longitude'], gee_df['Latitude']),
    crs="EPSG:4326"
)

# Load geocode table as GeoDataFrame
geocode_gdf = gpd.read_file(geocode_pth)

In [4]:

# Perform outer join based on Latitude/Longitude (gee) and lat (decimal)/long (decimal) (geocode)
merged_gdf = gee_gdf.merge(
    geocode_gdf,
    how='outer',
    left_on=['Latitude', 'Longitude'],
    right_on=['lat (decimal)', 'long (decimal)'],
    suffixes=('_gee', '_geocode')
)

In [5]:
merged_gdf

,system:index,CurrentlyM,Latitude,Longitude,SampleUID,Source,WesternHem,.geo,geometry_gee,Category (1=same as YFB lakes/2=different),...,CO2 analysis method,Original reference,Current reference (if applicable),additional notes,geocode_full_name,geocode_lat,geocode_lon,geocode_place_id,geocode_osm_id,geometry_geocode
0,0000000000000000037b,0.0,-60.00,-45.00,BML_00000000,BogardMapLakes,True,"{""type"":""Point"",""coordinates"":[-44.99999928308...",POINT (-45 -60),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,0000000000000000037c,0.0,24.29,60.23,BML_00000001,BogardMapLakes,False,"{""type"":""Point"",""coordinates"":[60.230000962811...",POINT (60.23 24.29),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,0000000000000000037d,0.0,25.03,60.26,BML_00000002,BogardMapLakes,False,"{""type"":""Point"",""coordinates"":[60.260001794699...",POINT (60.26 25.03),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,0000000000000000037e,0.0,25.11,61.15,BML_00000003,BogardMapLakes,False,"{""type"":""Point"",""coordinates"":[61.149998233057...",POINT (61.15 25.11),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,0000000000000000037f,0.0,25.25,61.28,BML_00000004,BogardMapLakes,False,"{""type"":""Point"",""coordinates"":[61.279998865169...",POINT (61.28 25.25),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9176,0000000000000000209d,0.0,82.92,-66.87,LRL_00001213,LitReviewLakes,True,"{""type"":""Point"",""coordinates"":[-66.86999929138...",POINT (-66.87 82.92),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
9177,000000000000000020a0,0.0,82.98,-75.42,LRL_00001216,LitReviewLakes,True,"{""type"":""Point"",""coordinates"":[-75.42000004699...",POINT (-75.42 82.98),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
9178,000000000000000020a1,0.0,82.98,-75.20,LRL_00001217,LitReviewLakes,True,"{""type"":""Point"",""coordinates"":[-75.20000137831...",POINT (-75.2 82.98),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
9179,0000000000000000209f,0.0,83.02,-75.39,LRL_00001215,LitReviewLakes,True,"{""type"":""Point"",""coordinates"":[-75.38999921510...",POINT (-75.39 83.02),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [14]:
# How many features were matched?
matched = merged_gdf[merged_gdf['geocode_full_name'].notnull()]
matched_count = len(matched)
matched_left_count = matched[matched['SampleUID'].notnull()].shape[0]
print(f"Number of matched features: {matched_count} / {merged_gdf.shape[0]}")
print(f"Number of matched features in og Bogard_lakes shp: {matched_count} / {matched_left_count}")

Number of matched features: 151 / 9181
Number of matched features in og Bogard_lakes shp: 151 / 151


In [13]:
matched

,system:index,CurrentlyM,Latitude,Longitude,SampleUID,Source,WesternHem,.geo,geometry_gee,Category (1=same as YFB lakes/2=different),...,CO2 analysis method,Original reference,Current reference (if applicable),additional notes,geocode_full_name,geocode_lat,geocode_lon,geocode_place_id,geocode_osm_id,geometry_geocode
13,00000000000000000388,1.0,45.89380,-73.96678,BML_00000013,BogardMapLakes,True,"{""type"":""Point"",""coordinates"":[-73.96677901367...",POINT (-73.96678 45.8938),2.0,...,equilibrator,Lapierre & del Giorgio 2012 JGRB J. Geophys. R...,Hastie et al. 2018. Glob. Change. Biol. 24: 71...,None,"Lac Connelly, Saint-Hippolyte, La Rivière-du-N...",45.897087,-73.963576,337621817.0,175457438.0,"POLYGON ((1696763.431 923144.283, 1696778.057 ..."
16,0000000000000000038b,1.0,45.91440,-74.04370,BML_00000016,BogardMapLakes,True,"{""type"":""Point"",""coordinates"":[-74.04369850684...",POINT (-74.0437 45.9144),2.0,...,equilibrator,Lapierre & del Giorgio 2012 JGRB J. Geophys. R...,Hastie et al. 2018. Glob. Change. Biol. 24: 71...,None,"Lac Fournelle, Saint-Hippolyte, La Rivière-du-...",45.913170,-74.037597,340697927.0,176694358.0,"POLYGON ((1691225.145 922635.86, 1691230.658 9..."
17,0000000000000000038c,1.0,45.92860,-74.03520,BML_00000017,BogardMapLakes,True,"{""type"":""Point"",""coordinates"":[-74.03519946023...",POINT (-74.0352 45.9286),2.0,...,equilibrator,Lapierre & del Giorgio 2012 JGRB J. Geophys. R...,Hastie et al. 2018. Glob. Change. Biol. 24: 71...,None,"Lac Morency, Saint-Hippolyte, La Rivière-du-No...",45.928364,-74.035979,339474072.0,176693097.0,"POLYGON ((1691181.462 924262.841, 1691189.56 9..."
18,0000000000000000038d,1.0,45.94296,-74.09010,BML_00000018,BogardMapLakes,True,"{""type"":""Point"",""coordinates"":[-74.09009991240...",POINT (-74.0901 45.94296),2.0,...,equilibrator,Lapierre & del Giorgio 2012 JGRB J. Geophys. R...,Hastie et al. 2018. Glob. Change. Biol. 24: 71...,None,"Lac Violon, Sainte-Adèle, Les Pays-d'en-Haut, ...",45.944119,-74.089041,337574371.0,176773269.0,"POLYGON ((1686819.491 924467.804, 1686845.222 ..."
20,0000000000000000038f,1.0,45.96069,-74.04157,BML_00000020,BogardMapLakes,True,"{""type"":""Point"",""coordinates"":[-74.04157151564...",POINT (-74.04157 45.96069),2.0,...,equilibrator,Lapierre & del Giorgio 2012 JGRB J. Geophys. R...,Hastie et al. 2018. Glob. Change. Biol. 24: 71...,None,"Lac du Pin Rouge, Saint-Hippolyte, La Rivière-...",45.960770,-74.040634,340225062.0,176773222.0,"POLYGON ((1689559.753 927546.271, 1689564.636 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,00000000000000000498,1.0,54.86770,-66.67668,BML_00000285,BogardMapLakes,True,"{""type"":""Point"",""coordinates"":[-66.67667942431...",POINT (-66.67668 54.8677),2.0,...,equilibrator,Bogard & del Giorgio 2016 GBC,None,None,"Lac Rivals, Lac-Vacher, Caniapiscau (MRC), Côt...",54.870175,-66.682065,350371340.0,2604487.0,"POLYGON ((1803992.338 2041603.204, 1804001.431..."
448,0000000000000000049b,1.0,54.90270,-66.88958,BML_00000288,BogardMapLakes,True,"{""type"":""Point"",""coordinates"":[-66.88957920411...",POINT (-66.88958 54.9027),2.0,...,equilibrator,Bogard & del Giorgio 2016 GBC,None,None,"Lac-Vacher, Caniapiscau (MRC), Côte-Nord, Québ...",54.854692,-66.939465,348010895.0,8449496.0,"POLYGON ((1768537.835 2039964.35, 1768552.305 ..."
455,000000000000000004a1,1.0,54.94872,-68.77256,BML_00000294,BogardMapLakes,True,"{""type"":""Point"",""coordinates"":[-68.77255947875...",POINT (-68.77256 54.94872),2.0,...,equilibrator,Bogard & del Giorgio 2016 GBC,None,None,"Lac Costebelle, Caniapiscau, Caniapiscau (MRC)...",54.949593,-68.772910,405594417.0,19060759.0,"POLYGON ((1679764.498 1994757.197, 1679777.661..."
487,000000000000000004b1,1.0,55.14247,-67.23103,BML_00000310,BogardMapLakes,True,"{""type"":""Point"",""coordinates"":[-67.23103040817...",POINT (-67.23103 55.14247),2.0,...,equilibrator,Bogard & del Giorgio 2016 GBC,None,None,"Lac Leroy, Rivière-Koksoak, ᑲᑎᕕᒃ Kativik, Nord...",